In [86]:
# Importings were changed

In [59]:
# suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

# from os import chdir
# chdir(r'C:\Users\Pia\OneDrive\Data Science\experiment_design\repo')

import pandas as pd
pd.options.display.max_rows = 250
import numpy as np
import csv
import glob
import random
import ntpath
import os
from functools import reduce

from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

## Majority Voting

### Load Train Data

In [61]:
# load stacking files
metadata_train = pd.read_csv(r'out/predictions_metadata_dev.csv', index_col = 0, header = 0)

textual_train = pd.read_csv(r'out/text_yhat_train.csv', index_col = 10, header = 0)
textual_train.index.names = ['movie']
textual_train = textual_train.drop('goodforairplanes', axis='columns')

visual_train = pd.read_csv(r'out/visual_predictions_train.csv', index_col = 0, header = 0)
#visual_train = visual_train.drop('goodforairplanes', axis='columns')

audio_train = pd.read_csv(r'out/audio_yhat_train.csv', index_col = 10, header = 0)
audio_train.index.names = ['movie']

# merge dataframes
#dfs = [metadata_train, textual_train, visual_train, audio_train]
dfs = [metadata_train, visual_train]
majority_train = reduce(lambda left, right: pd.merge(left, right, on = 'movie'), dfs)

# rearrange columns
majority_train = majority_train[[c for c in majority_train if c not in ['goodforairplanes']] + ['goodforairplanes']]
majority_train.head()

,decision_tree,knn,nearest_mean,logistic_regression,svm,bagging,random_forest,adaboost,gradient_boost,KNN,Decision Tree,Logistic Regression,SVM,Random Forest,AdaBoost,Gradient Boosting Tree,goodforairplanes
movie,,,,,,,,,,,,,,,,,
Seventh Son,0,1,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1
Welcome to Me,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,1,0
The Judge,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0
Transformers Age of Extinction,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0
The Normal Heart,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,1


### Load Test data

In [62]:
# load stacking files
metadata_test = pd.read_csv(r'out/predictions_metadata_test.csv', index_col = 0, header = 0)
metadata_test.index.names = ['movie']

textual_test = pd.read_csv(r'out/text_yhat_test.csv', index_col = 10, header = 0)
textual_test.index.names = ['movie']
textual_test = textual_test.drop('goodforairplanes', axis='columns')

visual_test = pd.read_csv(r'out/visual_predictions_test.csv', index_col = 0, header = 0)
#visual_test = visual_test.drop('goodforairplanes', axis='columns')

audio_test = pd.read_csv(r'out/audio_yhat_test.csv', index_col = 10, header = 0)
audio_test.index.names = ['movie']

# merge dataframes
#dfs = [metadata_test, textual_test, visual_test, audio_test]
dfs = [metadata_test, visual_test]
majority_test = reduce(lambda left, right: pd.merge(left, right, on = 'movie'), dfs)

# rearrange columns
majority_test = majority_test[[c for c in majority_test if c not in ['goodforairplanes']] + ['goodforairplanes']]
majority_test.head()

,decision_tree,knn,nearest_mean,logistic_regression,svm,bagging,random_forest,adaboost,gradient_boost,KNN,Decision Tree,Logistic Regression,SVM,Random Forest,AdaBoost,Gradient Boosting Tree,goodforairplanes
movie,,,,,,,,,,,,,,,,,
10.000 Km,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
12 Years a Slave,0,1,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1
21 Jump Street,0,1,1,1,1,0,0,0,0,0,0,1,1,0,0,0,1
2 States,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,0,1
Aanmodderfakker,1,0,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1


### Define Functions

In [160]:
def find_majority(li):
    
    myMap = {}
    maximum = ( '', 0 ) # (occurring element, occurrences)
    
    for element in li:
        
        if element in myMap: 
            myMap[element] += 1
            
        else: 
            myMap[element] = 1

        # Keep track of maximum on the go
        if myMap[element] > maximum[1]: maximum = (element, myMap[element])

    return maximum[0]

In [169]:
def majority_voting(predictions_df, CV = False):
    
    X = predictions_df.iloc[:, :-1]
    y = predictions_df.iloc[:,-1]
    
    if CV:
        print("WTF??")
        
    else:
        predictions = []
        
        for i in range(0,len(X.index)):
            
            li = X.iloc[i].tolist()
            predictions.append(find_majority(li))
            
        precision = precision_score(y, predictions)
        recall = recall_score(y, predictions)
        f1 = f1_score(y, predictions)
        
        scores = {"Precision": precision, "Recall": recall, "F1": f1}
        
    return scores

### Majority Voting on the Train Set
We apply Majority Voting on the training set without using cross-validation because cv would not make much sense here.

In [170]:
majority_scores_train = majority_voting(majority_train)
majority_scores_train

{'Precision': 0.7058823529411765,
 'Recall': 0.8421052631578947,
 'F1': 0.7679999999999999}

### Majority Voting on the Test Set

In [171]:
majority_scores_test = majority_voting(majority_test)
majority_scores_test

{'Precision': 0.6761904761904762, 'Recall': 0.8875, 'F1': 0.7675675675675676}

## Label Stacking

### Label Stacking on Training Set

In [85]:
def label_stacking_cv(df):

    log_reg = LogisticRegression(solver='lbfgs')  # solver='lbfgs' -> for not getting warnings

    X, y = df.iloc[:, :-1], df.iloc[:,-1]

    cv_scores = cross_validate(log_reg, X, y, cv=10, scoring=['precision', 'recall', 'f1'])

    print(np.mean(cv_scores['test_precision']), np.mean(cv_scores['test_recall']), np.mean(cv_scores['test_f1']))
    
label_stacking_cv(majority_train)
# label_stacking_cv(majority_test)

0.6422619047619047 0.67 0.638028638028638
0.6572755022755022 0.788888888888889 0.7117820621071396
